In [1]:
import face_rec

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: playground\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [2]:
import pandas as pd
import numpy as np
import cv2

In [3]:
face_rec.r.ping()

True

In [4]:
# face_rec.r.hdel('academy:register', 'Student@Ahikmah')
# face_rec.r.hdel('academy:register', 'Teacher@Aman Ansari')

#### 1. Extract data from db

In [5]:

retrieve_dict = face_rec.r.hgetall('academy:register')
retrieve_series = pd.Series(retrieve_dict)
retrieve_series = retrieve_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x: x.decode('utf-8'), index))
retrieve_series.index = index

retrieve_series

Student@Chris Evans           [0.7201715, -1.4684149, -0.7781253, -1.4601073...
Student@Scarlett Johansson    [0.19131963, -0.48843804, -1.690233, 0.4423967...
Teacher@Barack Obama          [0.94858104, -0.5584052, 0.094820715, -1.27965...
Teacher@Morgan Freeman        [0.48535103, -0.6192555, -0.4069932, 1.0687788...
Student@Awaliyatul Hikmah     [-0.82745886, -0.14807658, 1.2313854, 0.161325...
Student@Angelina Jolie        [0.43242034, 1.0748498, -0.12862365, -1.833291...
dtype: object

In [6]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['role_name', 'facial_features']
retrieve_df[['role', 'name']] = retrieve_df['role_name'].apply(lambda x: x.split('@')).apply(pd.Series)

retrieve_df

,role_name,facial_features,role,name
0,Student@Chris Evans,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",Student,Chris Evans
1,Student@Scarlett Johansson,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Student,Scarlett Johansson
2,Teacher@Barack Obama,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Teacher,Barack Obama
3,Teacher@Morgan Freeman,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",Teacher,Morgan Freeman
4,Student@Awaliyatul Hikmah,"[-0.82745886, -0.14807658, 1.2313854, 0.161325...",Student,Awaliyatul Hikmah
5,Student@Angelina Jolie,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",Student,Angelina Jolie


#### 2. Get Realtime Prediction

In [7]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if ret:
        # frame = cv2.flip(frame, 1)
        # frame = cv2.resize(frame, (640, 480))
        frame = face_rec.face_prediction(frame, retrieve_df, 'facial_features', thresh=0.5)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == 27: # press ESC to exit
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()